In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 6.3MB/s 
     |████████████████████████████████| 952kB 9.1MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=39d110e6a82d563a5b8a466ac5e95cf18e762499237c379ead7362419934eb73
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
!pip install cython
!pip install git+git://github.com/waspinator/coco.git@2.1.0

  Cloning git://github.com/waspinator/coco.git (to revision 2.1.0) to /tmp/pip-req-build-2rb7nsz3
  Running command git clone -q git://github.com/waspinator/coco.git /tmp/pip-req-build-2rb7nsz3
  Running command git checkout -q cd69b031a5dafdd40c0b2c52bd4a9c5d7f11382e
  Created wheel for pycocotools: filename=pycocotools-2.1-cp37-cp37m-linux_x86_64.whl size=280794 sha256=178add10267a52c3dd4f51958943b093d09fc997e978a065b2284c56afe83e1d
  Stored in directory: /tmp/pip-ephem-wheel-cache-scg300hb/wheels/42/68/4d/0ae439f3aece2e517244d3194de0db684e0e3d3b73bdbd5533
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


In [ ]:
!git clone https://github.com/waspinator/pycococreator.git

Cloning into 'pycococreator'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 165
Receiving objects: 100% (168/168), 180.92 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
%cd pycococreator/

/content/pycococreator


In [ ]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2
from pycococreatortools.pycococreatortools import create_annotation_info

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

plt.rcParams['axes.grid'] = False

In [ ]:
class CopyPaste():
    category_names = ['UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
    copy_image_list = dict()
    copy_mask_list = dict()
    none_object_image_list = []
    DIR_LIST = [[0, 1],
                [0, -1],
                [1, 0],
                [-1, 0]]
    
    def __init__(self, ann_file_path, dataset_path, copy_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], copy_image_num = 10, copy_per_image = 3):
        self.coco = COCO(ann_file_path)
        self.ann_file_path = ann_file_path
        self.dataset_path = dataset_path
        self.copy_labels = copy_labels
        self.get_copy_image()
        self.copy_image_num = copy_image_num

        if copy_per_image  > 3:
          copy_per_image = 3

        self.copy_per_image = copy_per_image

    def get_classname(self, classID, cats):
        for i in range(len(cats)):
            if cats[i]['id'] == classID:
                return cats[i]['name']
        return "None"

    def get_background_mask(self, image_info, image_id):
        image = cv2.imread(os.path.join(self.dataset_path, image_info["file_name"]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)

        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        ann_list = self.coco.loadAnns(ann_ids)

        cat_ids = self.coco.getCatIds()
        cats = self.coco.loadCats(cat_ids)

        mask = np.zeros((image_info["height"], image_info["width"]))
        
        biggest_bbox = [0, 0]
        smallest_bbox = [512, 512]

        for ann in ann_list:
          className = self.get_classname(ann['category_id'], cats)
          pixel_value = self.category_names.index(className)
          mask = np.maximum(self.coco.annToMask(ann) * pixel_value, mask)
          biggest_bbox[0] = max(biggest_bbox[0], ann["bbox"][2]) 
          biggest_bbox[1] = max(biggest_bbox[1], ann["bbox"][3])        
          smallest_bbox[0] = min(smallest_bbox[0], ann["bbox"][2]) 
          smallest_bbox[1] = min(smallest_bbox[1], ann["bbox"][3])
        
        mask = mask.astype(np.float32)
        return image, mask, biggest_bbox, smallest_bbox
    
    def copy_paste(self, image, mask, paste_image, paste_mask):
        used_position = []
        while len(used_position) < 500:
            start_h = random.randint(0, int(image.shape[0] - paste_image.shape[0]))
            start_w = random.randint(0, int(image.shape[1] - paste_image.shape[1]))
            if (start_h, start_w) in used_position:
                continue
            used_position.append((start_h, start_w))
            is_available = True
            roabst = 0
            for i in range(0, paste_mask.shape[0]):
              for j in range(0, paste_mask.shape[1]):
                if (paste_mask[i, j] > 0 and mask[i + start_h, j + start_w] > 0):
                    roabst += 1
                if roabst >= ((paste_image.shape[0] * paste_image.shape[1])/40):
                    is_available = False
                    break
              if not is_available:
                  break 
            if not is_available:
                continue
            
            seg_single_mask = np.zeros_like(mask)
            for i in range(0, paste_mask.shape[0]):
              for j in range(0, paste_mask.shape[1]):
                if paste_mask[i, j] > mask[i + start_h, j + start_w]:
                  image[i + start_h, j + start_w] = paste_image[i, j]
                  mask[i + start_h, j + start_w] = paste_mask[i, j]
                  seg_single_mask[i + start_h, j + start_w] = paste_mask[i, j]
            
          
            return image, mask, seg_single_mask, True
        print("[INFO] Cant paste image")
        return image, mask, None, False


    def copy_paste_image(self, image, mask, paste_list, maximum, minimum):
        did_it = False
        single_list = []
        for paste_img, (paste_mask, is_crowd), label in paste_list:
            size_h = random.randint(int(minimum[0]), int(maximum[0]))
            size_w = random.randint(int(minimum[1]), int(maximum[1]))

            transform = A.Compose([
                                    A.HorizontalFlip(p=0.5),
                                    A.VerticalFlip(p=0.5),
                                    A.Rotate(limit=270, p=0.6),
                                    A.Resize(size_h, size_w),
                                  ])

            transformed = transform(image=paste_img, mask=paste_mask)
            paste_img = transformed["image"]
            paste_mask = transformed["mask"]

            image, mask, single_temp_mask, tmp_did_it = self.copy_paste(image, mask, paste_img, paste_mask)
            if tmp_did_it:
              single_list.append((single_temp_mask, label, is_crowd))
            did_it |= tmp_did_it
        return image, mask, did_it, single_list

    def copy_paste_images(self):
        image_list = random.choices(range(0, len(self.none_object_image_list)), k= self.copy_image_num * 10)
        random.shuffle(image_list)

        paste_image_list = []#{"image": image, "mask": mask}
        for idx in tqdm(image_list):
            temp = self.none_object_image_list[idx]
            image_id, image_info = temp["image_id"], temp["image_info"]
            ann_list = temp["ann_list"]
            image, mask, biggest_bbox, smallest_bbox = self.get_background_mask(image_info, image_id)
            selected_label = [random.choice(self.copy_labels) for i in range(0, self.copy_per_image)]

            paste_list = []
            for label in selected_label:
              k = random.choice(range(0, len(self.copy_mask_list[label])))
              paste_list.append((self.copy_image_list[label][k], self.copy_mask_list[label][k], label))

            mean_bbox = [0, 0]
            mean_bbox[0] = (biggest_bbox[0] + smallest_bbox[0]) / 2
            mean_bbox[1] = (biggest_bbox[1] + smallest_bbox[1]) / 2

            image, mask , did_it, single_list = self.copy_paste_image(image, mask, paste_list, biggest_bbox, mean_bbox)
            if did_it:
              print(f"[INFO] Succeed: {len(paste_image_list)}/{self.copy_image_num}")
              paste_image_list.append((image, mask, image_info, ann_list, single_list)) 
            if len(paste_image_list) == self.copy_image_num:
              break
        return paste_image_list
    
    def get_copy_image(self):
        for idx in tqdm(range(0, len(self.coco.getImgIds())), desc="get_copy_image_masks"):
            image_id = self.coco.getImgIds(imgIds=idx)
            image_info = self.coco.loadImgs(ids=image_id)[0]

            image = cv2.imread(os.path.join(self.dataset_path, image_info["file_name"]))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
            
            ann_ids = self.coco.getAnnIds(imgIds=image_id)
            ann_list = self.coco.loadAnns(ann_ids)

            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            count = 0
            for i in range(len(ann_list)):
                copy_img = np.zeros_like(image)
                copy_mask = np.zeros((image_info["height"], image_info["width"]))
                className = self.get_classname(ann_list[i]['category_id'], cats)
                pixel_value = self.category_names.index(className)
                if pixel_value not in self.copy_labels:
                    continue
                count += 1
                copy_img = np.maximum(self.coco.annToMask(ann_list[i]).reshape((512, 512, -1)) * image, copy_img)
                copy_mask = np.maximum(self.coco.annToMask(ann_list[i]) * pixel_value, copy_mask)
                copy_img = copy_img.astype(np.float32)
                copy_mask = copy_mask.astype(np.float32)

                if pixel_value not in self.copy_image_list:
                    self.copy_image_list[pixel_value] = []
                    self.copy_mask_list[pixel_value] = []
                x_min, y_min, w, h = ann_list[i]["bbox"]
                x_max = int((x_min + w) * 1.2)
                y_max = int((y_min + h) * 1.2)
                x_min = int(x_min * 0.8)
                y_min = int(y_min * 0.8)

                copy_img = copy_img[y_min:y_max, x_min:x_max]
                copy_mask = copy_mask[y_min:y_max, x_min:x_max]

                self.copy_image_list[pixel_value].append(copy_img)
                self.copy_mask_list[pixel_value].append((copy_mask, ann_list[i]['iscrowd']))
            
            if count == 0:
                temp = {
                    "image_info":image_info,
                    "image_id":image_id,
                    "ann_list": ann_list
                }
                self.none_object_image_list.append(temp)


  

In [ ]:
cp = CopyPaste("/content/drive/MyDrive/boostcamp/stage_3_det/data/train.json","/content/drive/MyDrive/boostcamp/stage_3_det/data", [9], 100)

loading annotations into memory...
Done (t=7.80s)
creating index...
index created!


In [ ]:
new_images = cp.copy_paste_images()

In [ ]:
with open("/content/drive/MyDrive/boostcamp/stage_3_det/data/train.json", 'r') as f:
  ori_data = json.load(f)

In [ ]:
temp_coco = COCO("/content/drive/MyDrive/boostcamp/stage_3_det/data/train.json")

loading annotations into memory...
Done (t=5.23s)
creating index...
index created!


In [ ]:
image_dict_format = {
    'date_captured': None,
    'file_name': 'batch_01_vt/0003.jpg',
    'height': 512,
    'id': 0,
    'license': 0,
    'url': None,
    'width': 512
}

In [ ]:
import copy
new_data= copy.deepcopy(ori_data)

In [ ]:
start_image_idx = len(temp_coco.getImgIds())
start_ann_idx = len(temp_coco.getAnnIds())

In [ ]:
save_image_path = "/content/drive/MyDrive/boostcamp/stage_3_det/data/"

In [ ]:
from PIL import Image
from matplotlib.patches import Polygon, Rectangle

for i, new_image in enumerate(new_images):
    n_img = new_image[0]
    n_mask = new_image[1]
    origin_img_info = new_image[2] 
    ann_list = new_image[3]
    n_segs = new_image[4] 

    
    for ann in ann_list: 
        temp_ann = copy.deepcopy(ann)
        temp_ann["id"] = start_ann_idx
        temp_ann["image_id"] = (start_image_idx + i)
        new_data["annotations"].append(temp_ann)
        start_ann_idx += 1
    

    for segs, label, is_crowd in n_segs:
        ann = create_annotation_info(start_ann_idx, (start_image_idx + i), {"is_crowd" : is_crowd, "id": label}, segs/label , (512, 512))
        new_data["annotations"].append(ann)
        start_ann_idx +=1
    n_image_info = copy.deepcopy(image_dict_format)
    n_image_info["file_name"] = f"new_image/{i}.jpg"
    n_image_info["id"] = (start_image_idx + i)
    n_img = cv2.cvtColor(n_img, cv2.COLOR_BGR2RGB)
    #plt.imsave(os.path.join(save_image_path, n_image_info["file_name"]), n_img)
    cv2.imwrite(os.path.join(save_image_path, n_image_info["file_name"]), n_img)
    new_data["images"].append(n_image_info)    

In [ ]:
with open(os.path.join(save_image_path, "new_train_255_not_dev.json"), 'w') as f:
  json.dump(new_data, f, indent="\t")

In [ ]:
now_coco = COCO("/content/drive/MyDrive/boostcamp/stage_3_det/data/new_train_255_not_dev.json")

loading annotations into memory...
Done (t=4.24s)
creating index...
index created!


In [ ]:
ax.shape

(10, 2)

In [ ]:

fig, ax = plt.subplots(nrows=10, ncols=2, figsize=(12, 60))
origin_images_id =[]
for i in range(0, 20):
  origin_images_id.append(new_images[i][2]["id"])
  ax[i//2][i%2].imshow(new_images[i][0]/255.)
  ax[i//2][i%2].set_title(new_images[i][2]["id"])

  for ann in new_images[i][3]:
    color = 'red'
    x_min, y_min, w, h = ann["bbox"]
    rect = Rectangle((x_min, y_min), w, h, linewidth=3,edgecolor=color, facecolor='none', alpha=0.7, linestyle = '--')
    ax[i//2][i%2].add_patch(rect)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from matplotlib.patches import Polygon, Rectangle

fig, ax = plt.subplots(nrows=10, ncols=2, figsize=(12, 60))

for i in range(0, 20):
  image_id = now_coco.getImgIds(imgIds=origin_images_id[i])
  image_info = now_coco.loadImgs(ids=image_id)[0]

  image = cv2.imread(os.path.join(save_image_path, image_info["file_name"]))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
  image /= 255.0

  ax[i//2][i%2].imshow(image)

  ann_ids = now_coco.getAnnIds(imgIds=image_id)
  ann_list = now_coco.loadAnns(ann_ids)

  for ann in ann_list:
    color = 'red'
    x_min, y_min, w, h = ann["bbox"]
    rect = Rectangle((x_min, y_min), w, h, linewidth=3,edgecolor=color, facecolor='none', alpha=0.7, linestyle = '--')
    ax[i//2][i%2].add_patch(rect)


plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from matplotlib.patches import Polygon, Rectangle

fig, ax = plt.subplots(nrows=10, ncols=2, figsize=(12, 60))

for i in range(0, 20):
  image_id = now_coco.getImgIds(imgIds=(2617 + i))
  image_info = now_coco.loadImgs(ids=image_id)[0]

  image = cv2.imread(os.path.join(save_image_path, image_info["file_name"]))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
  image /= 255.0

  ax[i//2][i%2].imshow(image)

  ann_ids = now_coco.getAnnIds(imgIds=image_id)
  ann_list = now_coco.loadAnns(ann_ids)

  for ann in ann_list:
    color = 'red'
    x_min, y_min, w, h = ann["bbox"]
    rect = Rectangle((x_min, y_min), w, h, linewidth=3,edgecolor=color, facecolor='none', alpha=0.7, linestyle = '--')
    ax[i//2][i%2].add_patch(rect)


plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!tar -cvf battery_plus_data_not_dev255.tar /content/drive/MyDrive/boostcamp/stage_3_det/data/new_image

In [ ]:
!ls

battery_plus_data_not_dev255.tar  LICENSE	      README.md  setup.py
examples			  pycococreatortools  setup.cfg


In [ ]:
%mv battery_plus_data_not_dev255.tar /content/drive/MyDrive/boostcamp/stage_3_det/data/

In [ ]:
category_names = ["BackGround", 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

def get_classname(classID, cats):
        for i in range(len(cats)):
            if cats[i]['id'] == classID:
                return cats[i]['name']
        return "None"

for idx in range(0, 100):
    image_id = now_coco.getImgIds(imgIds=(2617 + idx))
    image_info = now_coco.loadImgs(ids=image_id)[0]

    image = cv2.imread(os.path.join(save_image_path, image_info["file_name"]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
            
    ann_ids = now_coco.getAnnIds(imgIds=image_id)
    ann_list = now_coco.loadAnns(ann_ids)

    cat_ids = now_coco.getCatIds()
    cats = now_coco.loadCats(cat_ids)

    mask = np.zeros((image_info["height"], image_info["width"]))
    
    for i in range(len(ann_list)):
        className = get_classname(ann_list[i]['category_id'], cats)
        pixel_value = category_names.index(className)
        
        mask = np.maximum(now_coco.annToMask(ann_list[i]) * pixel_value, mask)
    mask = mask.astype(np.float32)
    cv2.imwrite(os.path.join(save_image_path, f"new_mask/{idx}.png"), mask)     

In [ ]:
!tar -cvf battery_plus_data_not_dev255_mask.tar /content/drive/MyDrive/boostcamp/stage_3_det/data/new_mask

In [ ]:
%mv battery_plus_data_not_dev255_mask.tar /content/drive/MyDrive/boostcamp/stage_3_det/data/